In [1]:
import csv
import scipy.misc
import numpy as np
import matplotlib.pyplot as plt
import random 
import sklearn.utils
%matplotlib inline
import matplotlib


Load Paths of training data

In [2]:
with open('recs/driving_log.csv') as fd:
    reader = csv.reader(fd)
    samples = [line for line in reader]



Define Data augmentation functions

In [3]:
def brightness(image,angle):
    image =  matplotlib.colors.rgb_to_hsv(image)
    brightness = np.random.uniform()
    image[:,:,2] = image[:,:,2] * brightness
    image = matplotlib.colors.hsv_to_rgb(image)
    
    return image.astype(np.uint8),angle

def identity(image,angle):
    return image,angle

def flip(image, angle):
    image = np.fliplr(image)
    return image,-angle

functions = [brightness,identity,flip]

Define data split for training and validation

In [4]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.15)


Data generator

In [5]:
def generator(samples, batch_size=32):
    mask = scipy.misc.imread('mask.png')/255
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        random.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            angles = []
            for batch_sample in batch_samples:
                offset = [0,0.3, -0.3]
                for img_idx in range(3):
                    name = batch_sample[img_idx]
                    angle = float(batch_sample[3])
                    final_angle =angle+offset[img_idx]
                    image = scipy.misc.imread(name)
                    func = random.choice(functions)
                    image,final_angle=func(image,final_angle)
                    image= image*mask
                    image=image/255
                    images.append(image)
                    angles.append(final_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            
            yield  sklearn.utils.shuffle(X_train, y_train)


Define model

In [6]:
from keras.models import Model
from keras.layers import *
from keras import backend as K
from keras.regularizers import l2

def conv2d_bn(x, nb_filter, nb_row, nb_col,
              border_mode='same', subsample=(1, 1),
              name=None):
    '''Utility function to apply conv + BN.
    '''

    x = Convolution2D(nb_filter, nb_row, nb_col,
                      subsample=subsample,
                      activation='relu',
                      border_mode=border_mode,
                      init='he_normal',
                      W_regularizer=l2(0.001),)(x)
    if K.image_dim_ordering() == 'th':
        channel_axis = 1
    else:
        channel_axis = 3
    x = BatchNormalization(axis=channel_axis)(x)
    return x


def build_model(input_shape):
    img_input = Input(shape= input_shape, name='input' )
    x = Cropping2D(cropping=((40,20), (0,0)), input_shape=(160,320,3))(img_input)
    x = conv2d_bn(x,32,3,11,subsample=(1,1), border_mode='valid')
    x = MaxPooling2D()(x)
    x = conv2d_bn(x,64,3,11,subsample=(1,1), border_mode='valid')
    x = MaxPooling2D()(x)
    x = conv2d_bn(x,128,3,11,subsample=(1,1), border_mode='valid')
    x = MaxPooling2D()(x)
    x = conv2d_bn(x,32,3,11,subsample=(1,1), border_mode='valid')
    x = MaxPooling2D()(x)
    x = Flatten(name='Flatten')(x)
    x = Dropout(0.65)(x)
    x = Dense(256,  activation='tanh', )(x)
    x = Dropout(0.65)(x)
    pre = Dense(128,  activation='tanh', )(x)
    ang= Dense(1, activation='linear',name='Angle')(pre)
    model = Model(input=img_input, output=ang)
    return model

Using TensorFlow backend.


Build / Compile / Summary

In [7]:
model = build_model((160,320,3))

In [8]:
model.compile(loss='mse',optimizer='adam')

In [9]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input (InputLayer)               (None, 160, 320, 3)   0                                            
____________________________________________________________________________________________________
cropping2d_1 (Cropping2D)        (None, 100, 320, 3)   0           input[0][0]                      
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 98, 310, 32)   3200        cropping2d_1[0][0]               
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 98, 310, 32)   128         convolution2d_1[0][0]            
___________________________________________________________________________________________

Instanciate generators

In [10]:
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)




Train Model

In [11]:
model.fit_generator(train_generator,
                    samples_per_epoch= 3*len(train_samples),
                    validation_data=validation_generator,
                    nb_val_samples=3*len(validation_samples),
                    nb_epoch=7)

Epoch 1/7
21120/21120 [==============================] - 124s - loss: 0.7496 - val_loss: 0.5553
Epoch 2/7
21120/21120 [==============================] - 115s - loss: 0.5035 - val_loss: 0.3952
Epoch 3/7
21120/21120 [==============================] - 113s - loss: 0.3588 - val_loss: 0.3101
Epoch 4/7
21120/21120 [==============================] - 112s - loss: 0.2556 - val_loss: 0.2127
Epoch 5/7
21120/21120 [==============================] - 113s - loss: 0.1862 - val_loss: 0.1634
Epoch 6/7
21120/21120 [==============================] - 116s - loss: 0.1409 - val_loss: 0.1297
Epoch 7/7
21120/21120 [==============================] - 119s - loss: 0.1108 - val_loss: 0.1129


Save Model

In [12]:
model.save('model.h5')